In [ ]:
import os
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
import pylab
import pydicom as dicom
import glob
import png

In [ ]:
# Identificação dos caminhos dos diretório com imagens de treinamento e de validação

training_data = 'imagens/treinamento'
validation_data = 'imagens/validacao'

In [ ]:
# Transformação das imagens DICOM em PNG

diretorio_list = ['imagens/treinamento/positivo/', 'imagens/validacao/positivo/',
                  'imagens/treinamento/negativo/', 'imagens/validacao/negativo/', 'imagens/teste/']
for diretorio in diretorio_list:
    for img in glob.glob(diretorio + '*.dcm'):
        ImageFile = dicom.read_file(img) 
        png_nome = os.path.basename(img)[:-3] + 'png'
        pylab.imsave(diretorio+png_nome, ImageFile.pixel_array, cmap=pylab.cm.bone)
       

In [ ]:
# Geração das classes das imagens de treinamento e de validação

img_width, img_height = 512, 512

# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
        training_data,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')


In [ ]:
# Construção do modelo para a rede neural

model = Sequential()

# Primeira camada convolucional
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda camada convolucional
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Terceira camada convolucional
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
model.add(Flatten())

# Full connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# Treinamento e validação da rede

nb_epoch  = 40
nb_train_samples = 178
nb_validation_samples = 89

model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)
model.save_weights('modelos/basic_cnn_1.h5')

In [ ]:
# Teste

diretorio = 'imagens/teste/'
for img_path in glob.glob(diretorio + '*.png'):
    if '.png' in img_path:
        img = image.load_img(img_path, target_size=(512, 512))

        img_teste = image.img_to_array(img)
        img_teste = np.expand_dims(img_teste, axis=0) * 1./255

        nome_arquivo = os.path.basename(img_path)
        score = model.predict(img_teste)
        
        if score > 0.5:
            predicao = 'Com pólipo'
        else:
            predicao = 'Sem pólipo'
            
        print('ARQUIVO: ', nome_arquivo)
        print('Predição: ', predicao)
        print('Score: ', ': ', score, '\n')